### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.describe()

,movie_id,movie,genre
count,35479,35479,35230
unique,35479,35416,2736
top,0238660,Home (2016),Drama
freq,1,4,3602


In [21]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35479 entries, 0 to 35478
Data columns (total 3 columns):
movie_id    35479 non-null object
movie       35479 non-null object
genre       35230 non-null object
dtypes: object(3)
memory usage: 831.6+ KB


In [11]:
# cell for work
reviews.describe(include=np.object)

,user_id,movie_id,timestamp
count,863866,863866,863866
unique,67353,35479,851107
top,23751,1454468,1484827876
freq,2875,3086,38


In [12]:
# more work
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863866 entries, 0 to 863865
Data columns (total 4 columns):
user_id      863866 non-null object
movie_id     863866 non-null object
rating       863866 non-null int64
timestamp    863866 non-null object
dtypes: int64(1), object(3)
memory usage: 26.4+ MB


In [15]:
# and still more
reviews.describe()

,rating
count,863866.000000
mean,7.315878
std,1.853831
min,0.000000
25%,6.000000
50%,8.000000
75%,9.000000
max,10.000000


In [17]:
# and even more - open additional cells as necessary
pd.isna(reviews['rating']).sum()

0

In [26]:
# number of different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))
print(genres)

The number of genres is 28.
{'Sci-Fi', 'Musical', 'Short', 'Drama', 'Adult', 'News', 'Horror', 'Game-Show', 'Crime', 'Family', 'Western', 'Documentary', 'History', 'Film-Noir', 'Adventure', 'Sport', 'Action', 'Reality-TV', 'Thriller', 'Music', 'Talk-Show', 'Romance', 'Comedy', 'Biography', 'War', 'Mystery', 'Animation', 'Fantasy'}


In [27]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 35479,
'The number of ratings in the dataset': 863866,
'The number of different genres': 28,
'The number of unique users in the dataset':67353,
'The number missing ratings in the reviews dataset':0,
'The average rating given across all ratings':7.315878,
'The minimum rating given across all ratings':0,
'The maximum rating given across all ratings':10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [ ]:
# cell for work

In [ ]:
# and another

In [ ]:
# and a few more below, which you can use as necessary

In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()